In [1]:
import pandas as pd
import numpy as np
import torch
pd.set_option('display.max_columns', None) 
from modules.preprocessing import preprocess, preprocess_for_v2

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

uber_trips_correlation_test = uber_trips.drop(columns=['ride_id'])
numeric_cols = uber_trips_correlation_test.select_dtypes(include=[np.number]).columns
X = uber_trips_correlation_test[numeric_cols]

vif_data = pd.DataFrame()
vif_data["Feature"] = numeric_cols
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("Variance Inflation Factor (VIF):")
print(vif_data.sort_values("VIF", ascending=False))

print("\nCorrelation Matrix:")
correlation = X.corr()
print(correlation)

import matplotlib.pyplot as plt
plt.figure(figsize=(12, 10))
plt.imshow(correlation, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
plt.colorbar(label='Correlation')
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=45, ha='right')
plt.yticks(range(len(numeric_cols)), numeric_cols)
plt.title("Correlation Matrix - Multicollinearity Check")
plt.tight_layout()
plt.show()

In [ ]:

preprocessor, X, y, categorical_features, numerical_features = preprocess()